In [420]:
import pandas as pd
import numpy as np

In [421]:
RANDOM_SEED = 13022022

In [422]:
test = pd.read_csv('Task/test.csv')
test.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [423]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1614 entries, 0 to 1613
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Smiles  1614 non-null   object
dtypes: object(1)
memory usage: 12.7+ KB


In [424]:
test.sample(3)

,Smiles
89,O=C(NCCN1CCC(n2c(=O)[nH]c3ccccc32)CC1)c1cc2cc(...
481,C[C@]12CC[C@H]3[C@@H](C=CC4=CC(=O)CC[C@@]43C)[...
616,CO[C@@]1(NC(=O)C2SC(=C(C(N)=O)C(=O)O)S2)C(=O)N...


In [425]:
train = pd.read_csv('Task/train.csv')
train.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [426]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5557 entries, 0 to 5556
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Smiles  5557 non-null   object
 1   Active  5557 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 49.0+ KB


In [427]:
train.sample(3)

,Smiles,Active
4497,Clc1ccc2c(NCCNc3ccnc4cc(Cl)ccc34)ccnc2c1,False
1577,Cn1c(=O)c2c(ncn2CCO)n(C)c1=O,False
1964,CC(C)(C(=O)O)c1ccc(C(=O)CCCN2CCC(OC(c3ccccc3)c...,False


In [428]:
list(train['Active'].unique())

[False, True]

In [429]:
list_active = {False: 0, True: 1}

In [430]:
train['Active'] = train['Active'].map(list_active)

In [431]:
train.sample()

,Smiles,Active
5174,CN(C)c1ccc2c(c1)Sc1cc(N(C)C)ccc1N2.CS(=O)(=O)O...,0


In [432]:
train['train'] = 1
test['train'] = 0

In [433]:
df = pd.concat([train, test], ignore_index=True)

In [434]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7171 entries, 0 to 7170
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Smiles  7171 non-null   object 
 1   Active  5557 non-null   float64
 2   train   7171 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 168.2+ KB


In [435]:
df['train'].unique()

array([1, 0], dtype=int64)

In [436]:
df[df['train'] == 1]['Active'].value_counts()

0.00    5351
1.00     206
Name: Active, dtype: int64

In [437]:
from collections import Counter

atoms = Counter()
for i in df['Smiles'].unique():
    for j in i:
        atoms[j] += 1

In [438]:
atoms['C']

67272

In [439]:
new_columnes = list(atoms.keys())

In [440]:
print(new_columnes)

['C', 'O', 'c', '1', '2', '[', 'n', 'H', ']', '(', 'N', ')', '@', '.', 'l', '=', 'S', '3', '4', 'F', '-', 's', '/', '5', 'o', 'a', '+', '#', 'I', 'P', 'B', 'r', '\\', 'Z', '6', '7', '8', 'e', 'A', 'K', 'M', 'g', 'i', 'L', '9']


In [441]:
df['len_smiles'] = df['Smiles'].apply(lambda x: len(x))

In [442]:
df['len_smiles'].max()

707

In [443]:
df['len_smiles'].describe()

count   7171.00
mean      51.12
std       31.88
min        6.00
25%       34.00
50%       46.00
75%       60.00
max      707.00
Name: len_smiles, dtype: float64

In [444]:
df[df['len_smiles'] > 60]['Smiles'].count()

1732

In [445]:
df['smiles_list'] = df['Smiles'].apply(lambda x: list(x))

In [446]:
df['smiles_list'] = df['smiles_list'].apply(lambda x: count_i(x))

In [447]:
for i in new_columnes:
    df[i] = df['smiles_list'].apply(lambda x: x[i])

In [448]:
def count_i(x):
    atoms = Counter()
    for i in x:
        atoms[i] += 1
    return atoms

In [449]:
df.sample(3)

,Smiles,Active,train,len_smiles,smiles_list,C,O,c,1,2,...,7,8,e,A,K,M,g,i,L,9
2722,COc1cccc([C@@H](C)NCCCc2ccccc2Cl)c1.Cl,0.00,1,38,"{'C': 8, 'O': 1, 'c': 12, '1': 2, '(': 2, '[':...",8,1,12,2,2,...,0,0,0,0,0,0,0,0,0,0
3041,CCN1C(C)(C)CC(Oc2ccc(C(=O)Nc3ccc(NC(=O)Nc4cc(C...,0.00,1,73,"{'C': 17, 'N': 4, '1': 2, '(': 11, ')': 11, 'O...",17,3,14,2,2,...,0,0,0,0,0,0,0,0,0,0
5897,Oc1ccnc2cc(Cl)ccc12,nan,0,19,"{'O': 1, 'c': 9, '1': 2, 'n': 1, '2': 2, '(': ...",1,1,9,2,2,...,0,0,0,0,0,0,0,0,0,0


In [450]:
df.sample(3)

,Smiles,Active,train,len_smiles,smiles_list,C,O,c,1,2,...,7,8,e,A,K,M,g,i,L,9
4264,CC1(C)CCC(CN2CCN(c3ccc(C(=O)NS(=O)(=O)c4ccc(NC...,0.00,1,118,"{'C': 21, '1': 2, '(': 13, ')': 13, 'N': 5, '2...",21,7,25,2,4,...,0,0,0,0,0,0,0,0,0,0
6311,CN1CCN(CC(=O)N2c3ccccc3C(=O)Nc3cccnc32)CC1,nan,0,42,"{'C': 8, 'N': 4, '1': 2, '(': 3, '=': 2, 'O': ...",8,2,11,2,2,...,0,0,0,0,0,0,0,0,0,0
914,Nc1nc(Nc2ccc3c(c2)CC[C@@H](N2CCCC2)CC3)nn1-c1c...,0.00,1,67,"{'N': 3, 'c': 18, '1': 6, 'n': 5, '(': 4, '2':...",12,0,18,6,6,...,0,0,0,0,0,0,0,0,0,0


In [451]:
df.drop(['smiles_list'], axis = 1, inplace = True)

In [452]:
df.set_index(['Smiles'], inplace = True)

In [453]:
df.sample(3)

,Active,train,len_smiles,C,O,c,1,2,[,n,...,7,8,e,A,K,M,g,i,L,9
Smiles,,,,,,,,,,,,,,,,,,,,,
Cc1ccc(-c2ncccn2)c(C(=O)N2C[C@H](COc3ccc(F)cn3)CC[C@H]2C)c1,nan,0,59,9,2,15,2,4,2,3,...,0,0,0,0,0,0,0,0,0,0
CC[C@@]1(O)C(=O)OCc2c1cc1n(c2=O)Cc2cc3c(CN(C)C)c(O)ccc3nc2-1.Cl,0.00,1,63,10,5,14,4,4,1,2,...,0,0,0,0,0,0,0,0,0,0
Cc1ccc2c(c1CCC(=O)O)CC[C@@H](NS(=O)(=O)c1ccc(Cl)cc1)C2,nan,0,54,9,4,12,4,2,1,0,...,0,0,0,0,0,0,0,0,0,0


In [454]:
train_new = df[df['train'] == 1]
test_new = df[df['train'] == 0]

In [455]:
train_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5557 entries, COc1ccc2[nH]cc(CCN)c2c1 to CCCCNc1ccc(C(=O)OCCN(C)C)cc1.Cl
Data columns (total 48 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Active      5557 non-null   float64
 1   train       5557 non-null   int64  
 2   len_smiles  5557 non-null   int64  
 3   C           5557 non-null   int64  
 4   O           5557 non-null   int64  
 5   c           5557 non-null   int64  
 6   1           5557 non-null   int64  
 7   2           5557 non-null   int64  
 8   [           5557 non-null   int64  
 9   n           5557 non-null   int64  
 10  H           5557 non-null   int64  
 11  ]           5557 non-null   int64  
 12  (           5557 non-null   int64  
 13  N           5557 non-null   int64  
 14  )           5557 non-null   int64  
 15  @           5557 non-null   int64  
 16  .           5557 non-null   int64  
 17  l           5557 non-null   int64  
 18  =           5557 non-nul

In [456]:
X = train_new.drop(['Active', 'train', 'len_smiles'], axis = 1)
y = train_new['Active']

In [457]:
from sklearn import model_selection, datasets, metrics, tree 

In [458]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(X, y, 
                                                                                    test_size=0.30, random_state=42)

In [459]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

In [460]:
clf = tree.DecisionTreeClassifier()
clf.fit(train_data, train_labels)

DecisionTreeClassifier()

In [461]:
predictions_clf = clf.predict(test_data)
metrics.f1_score(test_labels, predictions_clf)

0.10144927536231885

In [462]:
bagging_trees = BaggingClassifier(clf, n_estimators = 150)
bagging_trees.fit(train_data,train_labels)
predictions_bg = bagging_trees.predict(test_data)

In [463]:
metrics.f1_score(test_labels, predictions_bg)

0.05128205128205128

In [464]:
from lazypredict.Supervised import LazyClassifier

In [465]:
clf_lc = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf_lc.fit(train_data, test_data, train_labels, test_labels)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:14<00:00,  2.01it/s]


In [466]:
print(models)

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
NearestCentroid                    0.76               0.54     0.54      0.83   
ExtraTreeClassifier                0.93               0.53     0.53      0.93   
DecisionTreeClassifier             0.93               0.53     0.53      0.93   
LGBMClassifier                     0.96               0.53     0.53      0.94   
LabelSpreading                     0.94               0.52     0.52      0.93   
LabelPropagation                   0.94               0.52     0.52      0.93   
BaggingClassifier                  0.95               0.52     0.52      0.94   
ExtraTreesClassifier               0.96               0.51     0.51      0.94   
SGDClassifier                      0.95               0.51     0.51      0.94   
LinearDiscriminantAnalysis         0.95               0.51     0.51      0.94   
Perceptron                  

In [467]:
from sklearn.neighbors import NearestCentroid

In [468]:
NC_model = NearestCentroid()

In [469]:
train_data, test_data, train_labels, test_labels
NC_model.fit(train_data, train_labels)
predictions_clf_tree = NC_model.predict(test_data)
metrics.f1_score(test_labels, predictions_clf_tree)

0.09809809809809808

In [470]:
test_new_data = test_new.drop(['Active', 'train', 'len_smiles'], axis = 1) 

In [471]:
predictions = clf.predict(test_new_data)

In [472]:
test_new_data['Active'] = predictions

In [473]:
test_new_data.sample(5)

,C,O,c,1,2,[,n,H,],(,...,8,e,A,K,M,g,i,L,9,Active
Smiles,,,,,,,,,,,,,,,,,,,,,
O=C(O)CSc1nnc(Br)n1-c1ccc(C2CC2)c2ccccc12,5,2,12,4,4,0,3,0,0,3,...,0,0,0,0,0,0,0,0,0,1.00
COc1ccc(C(=O)Nc2c(Cl)cncc2Cl)cc1OC1CCCC1,9,3,11,4,2,0,1,0,0,3,...,0,0,0,0,0,0,0,0,0,1.00
O=C(NCc1ccc(C(F)(F)F)cc1)Nc1cccc2cnccc12,3,1,15,4,2,0,1,0,0,4,...,0,0,0,0,0,0,0,0,0,1.00
C=CCN(C)CCCCCCOc1ccc(C(=O)c2ccc(Br)cc2)c(F)c1,11,2,12,2,2,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0.00
c1cc(Cc2cnc[nH]2)ccn1,1,0,8,2,2,1,3,1,1,1,...,0,0,0,0,0,0,0,0,0,1.00


In [474]:
test_new_data.reset_index(inplace = True)

In [475]:
test_new_data.sample(5)

,Smiles,C,O,c,1,2,[,n,H,],...,8,e,A,K,M,g,i,L,9,Active
553,Cc1cc(O)cc2c1O[C@](C)(CCC[C@H](C)CCC[C@H](C)CC...,21,2,6,2,2,3,0,2,3,...,0,0,0,0,0,0,0,0,0,0.00
105,Clc1cccc(Nc2ncnc3ccccc23)c1,1,0,14,2,2,0,2,0,0,...,0,0,0,0,0,0,0,0,0,1.00
1313,CN(C)CCN1C(=O)c2cccc3cc(NC(N)=O)cc(c23)C1=O,7,3,10,2,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.00
828,COc1ccc(Nc2ncnc3ccc(OC)cc23)cc1,2,2,14,2,2,0,2,0,0,...,0,0,0,0,0,0,0,0,0,1.00
335,CN1N=C(N)c2cn([C@@H]3O[C@H](CO)[C@@H](O)[C@H]3...,7,4,6,2,2,4,3,4,4,...,0,0,0,0,0,0,0,0,0,0.00


In [476]:
submission = test_new_data[['Smiles', 'Active']]

In [477]:
submission.sample(4)

,Smiles,Active
950,O=C(O)c1ccc[n+]([O-])c1,1.00
85,O=C(O)CNC(=O)C1C(=O)N(C2CCCCC2)C(=O)N(C2CCCCC2...,0.00
1217,CCC1(CC)C(=O)C=CNC1=O,0.00
1458,Cc1c(C)c2c(c(C)c1O)CC[C@@](C)(CCC[C@H](C)CCC[C...,0.00


In [478]:
submission['Active'] = submission['Active'].apply(lambda x: int(x))

In [479]:
submission.to_csv('submission.csv', index=False)